In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import networkx as nx
import random
import copy

import sys
sys.path.append("Fall 2024")

import kmax as kx
import utility as util
from hierarchy import *
import z_solver as zs

In [56]:
def singular_ER_sample(n, lam):
    sum = 0 
    G = nx.erdos_renyi_graph(n, lam / n)
    for u,v in G.edges():
        G[u][v]['weight'] = np.random.uniform(0,1)
    ccs = [G.subgraph(c).copy() for c in nx.connected_components(G)]

    for cc in ccs:
        if cc.number_of_nodes() != 1:
            _, _, mdvp_val, _, _ = util.path_cover(cc, root = next(iter(cc)))
            sum += mdvp_val
    
    return sum / n

In [ ]:
print(singular_ER_sample(1000, 0.7))

0.15881792493267363
